In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

# --- FUNZIONE HELPER PER CARICARE I FONT IN SICUREZZA ---
def load_font(path, size, default_font=None):
    try:
        # Pillow supporta dimensioni float nelle versioni recenti
        return ImageFont.truetype(path, size)
    except OSError:
        print(f"ATTENZIONE: Font non trovato in '{path}'. Uso un default brutto.")
        return default_font if default_font else ImageFont.load_default()

def crea_locandina_v2(
    punteggio_casa, 
    punteggio_ospiti, 
    nome_team_casa, 
    nome_team_ospiti,
    nome_serie,             # Nuovo: es. "C2/B"
    giocatori_casa,         # Lista nomi
    giocatori_ospiti,       # Lista nomi
    risultati_match         # Lista risultati (es: "3-0", "1-3")
):
    print("--- Inizio creazione locandina ---")

    # Preserve original names for file naming (avoid embedded newlines)
    nome_team_casa_raw = nome_team_casa
    nome_team_ospiti_raw = nome_team_ospiti

    # 1. SCELTA DEL TEMPLATE PRINCIPALE (Vittoria vs Sconfitta)
    # Logica: Se punteggio casa > ospiti, vinciamo noi.
    if punteggio_casa > punteggio_ospiti:
        bg_path = "assets/templates/vittoria.png"
        print(f"Risultato {punteggio_casa}-{punteggio_ospiti}: VITTORIA. Carico sfondo verde.")
    else:
        bg_path = "assets/templates/sconfitta.png"
        print(f"Risultato {punteggio_casa}-{punteggio_ospiti}: SCONFITTA. Carico sfondo rosso.")

    try:
        base = Image.open(bg_path).convert("RGBA")
        W, H = base.size # Dimensioni totali immagine
    except FileNotFoundError:
        print(f"ERRORE FATALE: Non trovo lo sfondo principale: {bg_path}")
        return

    # 2. SETUP COLORI E FONT (Specifiche utente)
    TEXT_COLOR = "white" # Tutti i testi sono bianchi

    # Usiamo i percorsi e le dimensioni esatte richieste
    # N.B. Se non hai i file .ttf, il programma userà un font di default.
    font_nomi_bold = load_font("assets/fonts/Montserrat-ExtraBold.ttf", 27)
    font_serie_bold = load_font("assets/fonts/Montserrat-ExtraBold.ttf", 30)
    font_ris_match_bold = load_font("assets/fonts/Montserrat-ExtraBold.ttf", 40)
    font_ris_finale_anton = load_font("assets/fonts/Anton-Regular.ttf", 136)
    
    draw = ImageDraw.Draw(base)

    # --- INIZIO POSIZIONAMENTO ELEMENTI ---
    # NOTA SULLE COORDINATE: Uso ancore centrali (mm, lm, rm) per facilitare l'allineamento.
    # Modifica i valori X, Y per spostare gli elementi.

    # 3. SEZIONE HEADER (Nomi squadre, Serie, Punteggioone)
    # Ho stimato le coordinate Y guardando l'esempio, aggiustale tu.
    serie_pos = (W/2, 757)
    res_pos = (W/2, 897)
    squad1_pos = (221,759) # A sinistra del centro
    squad2_pos = (830,759) # A destra del centro
    # Nome Serie (es. C2/B) - Centrato in alto
    draw.text((serie_pos[0], serie_pos[1]), nome_serie, font=font_serie_bold, fill=TEXT_COLOR, anchor="mm")

    # Nomi Squadre
    # Casa (a sinistra del centro)
    #Controllo sulla lunghezza dei nomi delle squadre per evitare overflow (opzionale, dipende dal font)
    max_name_length = 20 # Puoi regolare questo valore in base alla dimensione del font e alla larghezza disponibile
    #Se il nome è troppo lungo, vado a capo (solo se il font supporta \n)
    if len(nome_team_casa) > max_name_length:
        nome_team_casa = nome_team_casa[:max_name_length] + "\n" + nome_team_casa[max_name_length:]
    if len(nome_team_ospiti) > max_name_length:
        nome_team_ospiti = nome_team_ospiti[:max_name_length] + "\n" + nome_team_ospiti[max_name_length:]

    #Se il nome va a capo, voglio l'allineamento orizzontale centrato 
    if "\n" in nome_team_casa:
        draw.text((squad1_pos[0], squad1_pos[1]), nome_team_casa, font=font_serie_bold, fill=TEXT_COLOR, anchor="mm", align="center")
    else : draw.text((squad1_pos[0], squad1_pos[1]), nome_team_casa, font=font_serie_bold, fill=TEXT_COLOR, anchor="mm")
    # Ospiti (a destra del centro)
    if "\n" in nome_team_ospiti: 
        draw.text((squad2_pos[0], squad2_pos[1]), nome_team_ospiti, font=font_serie_bold, fill=TEXT_COLOR, anchor="mm", align="center")
    else : draw.text((squad2_pos[0], squad2_pos[1]), nome_team_ospiti, font=font_serie_bold, fill=TEXT_COLOR, anchor="mm")

    # Punteggio Finale Gigante (es. 6 - 1) - Perfettamente centrato
    final_score_text = f"{punteggio_casa} - {punteggio_ospiti}"
    draw.text((res_pos[0], res_pos[1]), final_score_text, font=font_ris_finale_anton, fill=TEXT_COLOR, anchor="mm")

    # 4. LOGHI SOCIETÀ
    # Definisci centro e dimensione dei loghi
    size_logo = (180, 180) # Dimensione target
    pos_logo_ospiti_center = (754, 815)

    # Funzione helper per piazzare i loghi centrati
    def piazza_logo(path, pos, size, base_img):
        try:
            img = Image.open(path).convert("RGBA")            
            base_img.paste(img, (pos[0], pos[1]), mask=img)
        except FileNotFoundError:
            print(f"Logo non trovato: {path}")

    piazza_logo("assets/logos/cesenatico.png", pos_logo_ospiti_center, size_logo, base)


    # 5. SEZIONE INCONTRI INDIVIDUALI (Loop con Overlay Dinamici)
    print("--- Inizio elaborazione righe match ---")
    
    # Carica le immagini dei parallelepipedi
    try:
        overlay_win = Image.open("assets/overlays/win.png").convert("RGBA")
        overlay_lose = Image.open("assets/overlays/lose.png").convert("RGBA")
        overlay_dwin = Image.open("assets/overlays/dwin.png").convert("RGBA")
        overlay_dlose = Image.open("assets/overlays/dlose.png").convert("RGBA")
        # Assumiamo che win e lose abbiano le stesse dimensioni
        ow_w, ow_h = overlay_win.size
        ow_wd, ow_hd = overlay_dwin.size
    except FileNotFoundError:
        print("ERRORE FATALE: Mancano match_win.png o match_lose.png in assets/overlays/")
        return

    num_match = min(len(risultati_match), len(giocatori_casa), len(giocatori_ospiti))
    pos_overlay = [(142,990), (142,1034), (142,1081), (142,1125), (142,1209), (142,1254), (142,1301)]
    for i in range(num_match):

        risultato_str = risultati_match[i] # es. "3-0"

        # 5a. ANALISI VINCITORE DEL SINGOLO MATCH
        try:
            # Splitta "3-1" in ["3", "1"] e converte in numeri
            punti_casa_match, punti_ospiti_match = map(int, risultato_str.split("-"))
            # Se punti casa > punti ospiti, è vittoria per noi
            match_vinto_da_casa = punti_casa_match > punti_ospiti_match
        except ValueError:
            print(f"Errore formato risultato '{risultato_str}' alla riga {i}. Salto.")
            continue

        # 5b. SCEGLI E INCOLLA L'OVERLAY (Verde o Rosso)
        if i != 3: # Se non è il doppio, usa win/lose
            if match_vinto_da_casa:
                overlay_to_use = overlay_win
            else:
                overlay_to_use = overlay_lose
        
        else: # Se è il doppio, usa dwin/dlose
            if match_vinto_da_casa:
                overlay_to_use = overlay_dwin
            else:
                overlay_to_use = overlay_dlose
        
        # Calcola dove incollare (l'immagine overlay va centrata orizzontalmente e verticalmente rispetto alla riga)
        paste_x = pos_overlay[i][0]
        paste_y = pos_overlay[i][1]
        current_y_center = paste_y + (ow_h/2 if i != 3 else ow_hd/2) # Centro verticale dell'overlay
        # Incolla la striscia colorata
        base.paste(overlay_to_use, (paste_x, paste_y), mask=overlay_to_use)


        # Giocatore Casa (Allineato a DESTRA verso il centro -> anchor="rm")
        # X = W/2 - un po' di spazio (es. 100px dal centro)
        draw.text((W/2 - 200, current_y_center), giocatori_casa[i], font=font_nomi_bold, fill=TEXT_COLOR, anchor="mm", align= "center")

        # Risultato (Perfettamente centrato -> anchor="mm")
        y_rerult = pos_overlay[i][1] + ow_h/2 if i != 3 else pos_overlay[i][1] + ow_hd/2
        draw.text((W/2, y_rerult), risultato_str, font=font_ris_match_bold, fill=TEXT_COLOR, anchor="mm")

        # Giocatore Ospiti (Allineato a SINISTRA dal centro -> anchor="lm")
        # X = W/2 + un po' di spazio
        draw.text((W/2 + 200, current_y_center), giocatori_ospiti[i], font=font_nomi_bold, fill=TEXT_COLOR, anchor="mm", align= "center")


    # 6. SALVATAGGIO FINALE
    os.makedirs("output", exist_ok=True)
    # Crea un nome file pulito
    clean_casa = nome_team_casa_raw.replace("\n", " ").replace(" ", "_").replace(".", "")
    clean_ospiti = nome_team_ospiti_raw.replace("\n", " ").replace(" ", "_").replace(".", "")
    nome_file_output = f"output/match_{clean_casa}_vs_{clean_ospiti}.png"
    
    base.save(nome_file_output)
    print(f"--- FINITO! Locandina salvata in: {nome_file_output} ---")

# --- BLOCCO DI TEST CON I DATI DELLA TUA IMMAGINE ---
if __name__ == "__main__":
    # Dati presi esattamente dalla tua immagine di esempio
    crea_locandina_v2(
        punteggio_casa=3,
        punteggio_ospiti=4,
        nome_team_casa="TT ACLI C2 LUGO",
        nome_team_ospiti="EVERPING CESENATICO PONENTE",
        nome_serie="C2/B",
        giocatori_casa=["Loboda D.", "Tabanelli A.", "Loboda J.", "Loboda J.\nPollino F.", "Loboda D.", "Loboda J.", "Tabanelli A."],
        # Nota: Ho aggiunto \n per il doppio per farlo andare a capo se il font lo supporta, altrimenti uscirà su una riga
        giocatori_ospiti=["Godio L.", "Castelvetro C.", "Montanari L.", "Castelvetro C.\nGodio L.", "Castelvetro C.", "Godio L.", "Montanari L."],
        risultati_match=["0-3", "0-3", "0-3", "3-0", "0-3", "3-1", "3-2"]
    )

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def avvia_browser():
    """Configura e avvia il browser Chrome."""
    chrome_options = Options()
    # chrome_options.add_argument("--headless") # Togli il commento per nascondere il browser
    chrome_options.add_argument("--start-maximized")
    
    # Inizializza il driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def naviga_e_scarica_dati(nome_serie, nome_squadra_casa, giornata_numero, is_ritorno=False):
    """
    Funzione principale che orchestra tutta la navigazione.
    Restituisce un dizionario con i dati pronti per la grafica.
    """
    driver = avvia_browser()
    wait = WebDriverWait(driver, 15)
    base_url = "https://portale.fitet.org/risultati/regioni/default_reg.asp?REG=9" # Emilia Romagna

    print(f"--- Avvio Scraping: {nome_serie} - Giornata {giornata_numero} ---")

    try:
        # 1. ACCESSO AL SITO
        driver.get(base_url)

        # 2. SELEZIONE SERIE (Frame: sommario2)
        print("Seleziono la serie...")
        wait.until(EC.frame_to_be_available_and_switch_to_it("sommario2"))
        print("Entrato nel frame 'sommario2' per la selezione serie.")
        
        # Cerca il link che contiene il nome della serie (es. "C2/B")
        serie_link = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, nome_serie)))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", serie_link)
        time.sleep(1) # Piccola pausa per stabilizzare lo scroll
        serie_link.click()
        
        driver.switch_to.default_content() # Torna al livello base

        # 3. SELEZIONE CALENDARIO (Frame: header)
        print("Apro il calendario...")
        wait.until(EC.frame_to_be_available_and_switch_to_it("principale3"))
        print("Entrato nel frame 'principale3' per il calendario.")
        wait.until(EC.frame_to_be_available_and_switch_to_it("header"))
        print("Entrato nel frame 'header' per il calendario.")
        
        calendario = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Calendario incontri")))
        print("Trovato il link del calendario.")
        time.sleep(1) # Piccola pausa per stabilizzare lo scroll
        calendario.click()
        
        driver.switch_to.default_content()

        # 4. RICERCA MATCH NEL CALENDARIO (Frame: corpo)
        print(f"Cerco il match della giornata {giornata_numero}...")
        wait.until(EC.frame_to_be_available_and_switch_to_it("principale3"))
        print("Entrato nel frame 'principale3' per la ricerca del match.")
        wait.until(EC.frame_to_be_available_and_switch_to_it("corpo"))
        print("Entrato nel frame 'corpo' per la ricerca del match.")
        
        # XPATH Dinamico: Cerca la tabella che contiene "X^ Giornata"
        # Nota: Sul sito spesso scrivono "1^ Giornata", "2^ Giornata" etc.
        header_giornata = f"{giornata_numero}^ Giornata"
        
        # Trova la riga (TR) che contiene l'intestazione della giornata
        # Poi cerca la tabella padre o le righe successive
        #Voglio scegliere la tabella che è uguale al numero di giornata
        righe = driver.find_elements(By.XPATH, f"//div/table[{giornata_numero+1}]/tbody/tr")
        match_found = False
        
        for riga in righe:
            testo = riga.text
            # Controlliamo se siamo nella sezione della giornata giusta (logica semplificata)
            # Qui cerchiamo direttamente la riga che ha la squadra e un punteggio
            if nome_squadra_casa in testo:
                colonne = riga.find_elements(By.TAG_NAME, "td")
                if len(colonne) > 5:
                    # Logica colonne (basata sulla tua analisi):
                    # Andata (is_ritorno=False) -> Punteggio in colonna index 2 (3° td)
                    # Ritorno (is_ritorno=True) -> Punteggio in colonna index 5 (6° td)
                    idx = 5 if is_ritorno else 2
                    
                    try:
                        link_punteggio = colonne[idx].find_element(By.TAG_NAME, "a")
                        # Verifica che il link contenga un punteggio (es "5-2")
                        if "-" in link_punteggio.text:
                            print(f"Match trovato")
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link_punteggio)
                            link_punteggio.click()
                            match_found = True
                            break
                    except:
                        continue
        
        if not match_found:
            raise Exception("Match non trovato o punteggio non ancora inserito.")

        driver.switch_to.default_content()

        # 5. ESTRAZIONE DATI REFERTO
        print("Leggo i dettagli del referto...")
        
        # Navigazione profonda nei frame come descritto
        driver.switch_to.default_content()
        wait.until(EC.frame_to_be_available_and_switch_to_it("principale3"))
        wait.until(EC.frame_to_be_available_and_switch_to_it("corpo"))
        wait.until(EC.frame_to_be_available_and_switch_to_it("header"))
        
        # --- A. Nomi Squadre e Punteggio Totale (Dal primo DIV) ---
        # Percorso: Div 1 -> Tabella -> TR 1 -> TD 1 (Nomi) e TD 2 (Punti)
        div1 = driver.find_element(By.XPATH, "//div[1]")
        div2 = driver.find_element(By.XPATH, "//div[2]") # Per i dettagli dei match più avanti
        # Tabella Nomi (nella prima TD del Div 1)
        n1 = div1.find_element(By.XPATH, ".//table/tbody/tr[1]/td[1]/table/tbody/tr[1]/td[1]").text.strip() # Squadra 1
        n2 = div1.find_element(By.XPATH, ".//table/tbody/tr[1]/td[1]/table/tbody/tr[1]/td[3]").text.strip() # Simbolo "vs" o simile
        print(f"Simbolo tra squadre: '{n2}'")
        #n2 = div2.find_element(By.XPATH, ".//table/tbody/tr[1]/td[3]").text.strip() # Squadra 2
        # Tabella Punteggi (nella seconda TD del Div 1)
        tab_punti = div1.find_element(By.XPATH, ".//table/tbody/tr[1]/td[2]/table/tbody")
        p1 = tab_punti.find_element(By.XPATH, ".//tr[1]/td[1]").text.strip() # Punti Sq 1
        p2 = tab_punti.find_element(By.XPATH, ".//tr[1]/td[3]").text.strip() # Punti Sq 2

        # Identifichiamo chi siamo noi per assegnare correttamente Casa/Ospite sulla locandina
        if nome_squadra_casa.upper() in n1.upper():
            nome_avversario, p_casa, p_ospiti = n2, p1, p2
            siamo_squadra_1 = True
        else:
            nome_avversario, p_casa, p_ospiti = n1, p2, p1
            siamo_squadra_1 = False

        # --- B. Dettaglio Singoli Match (Dal secondo DIV) ---
        giocatori_casa = []
        giocatori_ospiti = []
        risultati_match = []
        
        # Prendiamo tutte le righe della tabella nel secondo DIV, saltando l'intestazione
        righe_match = driver.find_elements(By.XPATH, "//div[2]/table/tbody/tr")[1:]

        for riga in righe_match:
            cols = riga.find_elements(By.TAG_NAME, "td")
            s1_player = cols[1].text.strip()
            s2_player = cols[2].text.strip()
            
            # SV = Set Vinti da Squadra 1 | SP = Set Persi da Sq 1 (quindi vinti da Sq 2)
            sv = cols[13].text.strip() 
            sp = cols[14].text.strip()

            if siamo_squadra_1:
                giocatori_casa.append(s1_player)
                giocatori_ospiti.append(s2_player)
                risultati_match.append(f"{sv}-{sp}")
            else:
                giocatori_casa.append(s2_player)
                giocatori_ospiti.append(s1_player)
                risultati_match.append(f"{sp}-{sv}")

        print(f"Estrazione completata: {nome_squadra_casa} vs {nome_avversario} ({p_casa}-{p_ospiti})")
        
        return {
            "punteggio_casa": int(p_casa),
            "punteggio_ospiti": int(p_ospiti),
            "nome_team_casa": nome_squadra_casa,
            "nome_team_ospiti": nome_avversario,
            "nome_serie": nome_serie,
            "giocatori_casa": giocatori_casa,
            "giocatori_ospiti": giocatori_ospiti,
            "risultati_match": risultati_match
        }

    except Exception as e:
        print(f"ERRORE CRITICO: {e}")
        return None
    finally:
        driver.quit()

# --- BLOCCO DI TEST ---
if __name__ == "__main__":
    # Esempio di utilizzo
    dati = naviga_e_scarica_dati(
        nome_serie="D1/C", 
        nome_squadra_casa="TT ACLI D1 ALFA LUGO", 
        giornata_numero=3,   # Metti la giornata corretta che ha un risultato inserito
        is_ritorno=False       # False = Andata, True = Ritorno
    )
    print("Dati Scaricati:", dati)

In [ ]:
# from scraper import naviga_e_scarica_dati
# from generator_v2 import crea_locandina_v2

# --- CONFIGURAZIONE UTENTE ---
# Modifica questi parametri prima di lanciare il programma
SERIE = "C2/B"
MIA_SQUADRA = "TT ACLI C2 LUGO"
GIORNATA = 3           # Inserisci il numero della giornata
RITORNO = False        # False = Andata, True = Ritorno

def main():
    print("==========================================")
    print(f"🏓  GENERATORE AUTOMATICO LOCANDINE FITET")
    print("==========================================")
    print(f"Squadra: {MIA_SQUADRA}")
    print(f"Campionato: {SERIE} - Giornata {GIORNATA} ({'Ritorno' if RITORNO else 'Andata'})")
    print("------------------------------------------")

    # 1. AVVIO LO SCRAPER
    print("⏳ 1. Avvio il browser e cerco i risultati...")
    try:
        # Nota: Convertiamo GIORNATA in int per sicurezza
        dati_partita = naviga_e_scarica_dati(SERIE, MIA_SQUADRA, int(GIORNATA), is_ritorno=RITORNO)
    except Exception as e:
        print(f"❌ Errore imprevisto durante lo scaricamento: {e}")
        dati_partita = None

    # 2. CONTROLLO E GENERAZIONE GRAFICA
    if dati_partita:
        print("\n✅ Dati scaricati con successo!")
        print(f"   Match: {dati_partita['nome_team_casa']} vs {dati_partita['nome_team_ospiti']}")
        print(f"   Risultato: {dati_partita['punteggio_casa']} - {dati_partita['punteggio_ospiti']}")
        
        print("\n🎨 2. Creazione della locandina in corso...")
        try:
            crea_locandina_v2(
                punteggio_casa=dati_partita["punteggio_casa"],
                punteggio_ospiti=dati_partita["punteggio_ospiti"],
                nome_team_casa=dati_partita["nome_team_casa"],
                nome_team_ospiti=dati_partita["nome_team_ospiti"],
                nome_serie=dati_partita["nome_serie"],
                giocatori_casa=dati_partita["giocatori_casa"],
                giocatori_ospiti=dati_partita["giocatori_ospiti"],
                risultati_match=dati_partita["risultati_match"]
            )
            print("\n🎉 TUTTO FATTO! Controlla la cartella 'output'.")
        except Exception as e:
             print(f"❌ Errore durante la generazione grafica: {e}")
    else:
        print("\n❌ ERRORE: Non sono stati trovati dati validi per questa giornata.")
        print("   Suggerimenti:")
        print("   - Controlla che il risultato sia già stato caricato sul portale.")
        print("   - Verifica se è una giornata di Andata (RITORNO = False) o Ritorno.")
        print("   - Controlla il numero della giornata.")

if __name__ == "__main__":
    main()

🏓  GENERATORE AUTOMATICO LOCANDINE FITET
Squadra: TT ACLI C2 LUGO
Campionato: C2/B - Giornata 3 (Ritorno)
------------------------------------------
⏳ 1. Avvio il browser e cerco i risultati...
--- Avvio Scraping: C2/B - Giornata 3 ---
Seleziono la serie...
Entrato nel frame 'sommario2' per la selezione serie.
Apro il calendario...
Entrato nel frame 'principale3' per il calendario.
Entrato nel frame 'header' per il calendario.
Trovato il link del calendario.
Cerco il match della giornata 3...
Entrato nel frame 'principale3' per la ricerca del match.
Entrato nel frame 'corpo' per la ricerca del match.
Match trovato
Leggo i dettagli del referto...
Simbolo tra squadre: 'TT ACLI C2 LUGO'
Estrazione completata: TT ACLI C2 LUGO vs ASS. DYNAMIS TOPSOLID "A" (-)
ERRORE CRITICO: invalid literal for int() with base 10: ''

❌ ERRORE: Non sono stati trovati dati validi per questa giornata.
   Suggerimenti:
   - Controlla che il risultato sia già stato caricato sul portale.
   - Verifica se è una g